In [1]:
import mne
import numpy as np


Открываем запись ЭЭГ

In [2]:
data = mne.io.read_raw_edf("data/record0-Copy1.edf")
raw_data = data.get_data()
# you can get the metadata included in the file and a list of all channels:
info = data.info
channels = data.ch_names

Extracting EDF parameters from /home/gromov_n/extreme-events/EEG seizure/EpiActivity/data/record0-Copy1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


/tmp/ipykernel_3974060/659960048.py:1: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  data = mne.io.read_raw_edf("data/record0-Copy1.edf")


In [15]:
info

Measurement date,"January 28, 2024 21:54:45 GMT"
Experimenter,Unknown
Participant,79CA7028-6DBC-43A2-80C4-7D616BAD34C9
Digitized points,Not available
Good channels,20 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,199.50 Hz
Highpass,3.00 Hz
Lowpass,70.00 Hz


Собираю аннотацию, в формате списка из трех массивов [все времена старта события в секундах, все длительности в секундах, все описания] длительности все должны быть >=0, в edf бывает -1, тут нужно переделывать в 0. Чтобы перевести в секунды, нужно просто поделить на частоту дискретизации, она хранится в переменной info: info['sfreq']

In [26]:
dur_max = preds_annotations[1].max()
preds_annotations = list(preds_annotations)
preds_annotations[1] = np.clip(preds_annotations[1], a_min=0, a_max = dur_max)
preds_annotations[1].min()

0.0

Склеиваю со старой аннотацией

In [28]:
old_annot = np.array([[data.annotations[i]["onset"], data.annotations[i]["duration"], data.annotations[i]["description"]] 
                      for i in range(len(data.annotations))])
full_annot = np.concatenate([np.array(preds_annotations), old_annot.T], axis=1)
annotations = mne.Annotations(full_annot[0], full_annot[1], full_annot[2]) # тот же формат аргументов

Добавляю в запись

In [29]:
data.set_annotations(annotations)
data.annotations

<Annotations | 3948 segments: * (2953), EndPredictionTime (1), ...>

In [4]:
print(info)

<Info | 8 non-empty values
 bads: []
 ch_names: EEG Fp1-F3, EEG Fp2-F4, EEG F3-C3, EEG F4-C4, EEG C3-P3, EEG ...
 chs: 20 EEG
 custom_ref_applied: False
 highpass: 3.0 Hz
 lowpass: 70.0 Hz
 meas_date: 2024-01-28 21:54:45 UTC
 nchan: 20
 projs: []
 sfreq: 199.5 Hz
 subject_info: 5 items (dict)
>


In [ ]:
data.set

Сохраняю

In [3]:
data.export("predicts/Test.edf", overwrite=True)

Reading 0 ... 1551204  =      0.000 ...  7775.352 secs...


/tmp/ipykernel_3974060/2747206715.py:1: RuntimeWarning: Data has a non-integer sampling rate of 199.50274692224406; writing to EDF format may cause a small change to sample times.
  data.export("predicts/Test.edf", overwrite=True)


In [7]:
durs = [round(len(s) / info["sfreq"], 12) for s in raw_data]
durs

[7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566,
 7775.3566]

In [7]:
raw_data[-2].shape

(1551205,)